In [42]:
import ccxt
import numpy as np
import pandas as pd

import tensorflow as tf

# import add_all_ta_features
from ta import add_all_ta_features

# import tensorflow as tf
import matplotlib.pyplot as plt

# fetch btc ohlcv

In [43]:
data = pd.read_csv('btc.csv')
data['date'] = pd.to_datetime(data['timestamp'], unit='ms')

In [44]:

# train test split
train_size = int(len(data) * 0.80)
test_size = len(data) - train_size
# TypeError: list indices must be integers or slices, not tuple
train, test = data[0:train_size], data[train_size:len(data)]


In [45]:
def create_features(df):
    # add all indicators with add_all_ta_features
    df = add_all_ta_features( df, open="open", high="high", low="low", close="close", volume="volume", fillna=True)
    return df


In [46]:
ohlcv = create_features(data)

c:\Users\MrBios\AppData\Local\Programs\Python\Python39\lib\site-packages\ta\trend.py:730: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  self._trs[0] = diff_directional_movement.dropna()[
c:\Users\MrBios\AppData\Local\Programs\Python\Python39\lib\site-packages\ta\trend.py:748: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  self._dip[0] = pos.dropna()[0: self._window].sum()
c:\Users\MrBios\AppData\Local\Programs\Python\Python39\lib\site-packages\ta\trend.py:760: FutureWarning: The b

In [47]:
def create_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(ohlcv.shape[1], 1)))
    model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(50, activation='relu'))
    model.add(tf.keras.layers.Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

model = create_model()
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 92, 64)            192       
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 46, 64)           0         
 1D)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 2944)              0         
                                                                 
 dense_8 (Dense)             (None, 50)                147250    
                                                                 
 dense_9 (Dense)             (None, 1)                 51        
                                                                 
Total params: 147,493
Trainable params: 147,493
Non-trainable params: 0
________________________________________________

In [49]:
history = model.fit(ohlcv,  epochs=10, batch_size=32, verbose=1, shuffle=False)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [ ]:
# plot loss and accuracy
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
# accuracy
plt.plot(history.history['accuracy'], label='train')
plt.legend()
plt.show()
